In [1]:
!pip install nltk streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.2 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=f57d8e66610c145a9dd0895d013bbe9b9d39c5f310782462e331d91b297d560b
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzloc

In [26]:
import nltk
import string
import streamlit as st

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [17]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

#### Prétraitement des données que le ChatBot utilisera pour générer des réponses

In [18]:
# Chargement du fichier texte
with open('world_war_event.txt', 'r', encoding='utf-8') as f:
    data = f.read().replace('\n', ' ')

# Tokeniser le texte en phrases
sentences = sent_tokenize(data)

- La tokenisation est une étape fondamentale du traitement du langage naturel qui consiste à diviser un texte en unités discrètes appelées "tokens". Un token peut être un mot, une phrase, un caractère ou même un symbole, selon le niveau de granularité souhaité.

- La lemmatisation est une technique linguistique utilisée dans le traitement du langage naturel pour normaliser les mots en les ramenant à leur forme de base, appelée "lemme". Le lemme représente la forme canonique ou le dictionnaire d'un mot. Par exemple, le lemme du mot "courir" est "courir", du mot "voitures" est "voiture

In [22]:
# Fonction de Prétraitement des phrases
def preprocess(sentence):
    words = word_tokenize(sentence) # Tokenisation les phrases en mots
    # Suppression des stopwords et des ponctuations
    words = [word.lower() for word in words if word.lower() not in stopwords.words('english') and word not in string.punctuation]
    # Lemmatisation des mots
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return words

# Prétraitrement des phrases dans le texte
corpus = [preprocess(sentence) for sentence in sentences]

- Calcul de la similarité entre la requête prétraitée et la phrase courante en utilisant le coefficient de similarité de Jaccard. <br>Le coefficient de similarité de Jaccard est une mesure de similarité entre deux ensembles et est défini comme la taille de l'intersection divisée par la taille de l'union des ensembles. Dans ce cas, nous traitons la requête prétraitée et chaque phrase du corpus comme des ensembles de mots et calculons leur coefficient de similarité Jaccard.

In [27]:
def get_most_relevant_sentence(query):
  """
    Cette fonction renvoie la phrase la plus pertinente dans un corpus en fonction de la requête de l'utilisateur.

    Args:
        query (str): Requête de l'utilisateur

    Returns:
        str: Phrase pertinante.
  """
  query = preprocess(query)
  max_similarity = 0
  most_relevant_sentence = ""
  for sentence in corpus:
      similarity = len(set(query).intersection(sentence)) / float(len(set(query).union(sentence)))
      if similarity > max_similarity:
          max_similarity = similarity
          most_relevant_sentence = " ".join(sentence)
  return most_relevant_sentence


def chatbot(question):
  """
    Cette fonction prend la question de l'utilisateur et renvoie la réponse appropriée.

    Args:
        question (str): RequQuestion de l'utilisateur

    Returns:
        str: Réponse su ChatBot.
  """
  most_relevant_sentence = get_most_relevant_sentence(question)
  return most_relevant_sentence

## Streamlit App

In [28]:
%%writefile app.py
# importation des librairies necessaires
import nltk
import string
import streamlit as st
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Chargement du fichier texte
with open('world_war_event.txt', 'r', encoding='utf-8') as f:
    data = f.read().replace('\n', ' ')

# Tokeniser le texte en phrases
sentences = sent_tokenize(data)

# Fonction de Prétraitement des phrases
def preprocess(sentence):
    words = word_tokenize(sentence) # Tokenisation les phrases en mots
    # Suppression des stopwords et des ponctuations
    words = [word.lower() for word in words if word.lower() not in stopwords.words('english') and word not in string.punctuation]
    # Lemmatisation des mots
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return words

# Prétraitrement des phrases dans le texte
corpus = [preprocess(sentence) for sentence in sentences]

def get_most_relevant_sentence(query):
  """
    Cette fonction renvoie la phrase la plus pertinente dans un corpus en fonction de la requête de l'utilisateur.

    Args:
        query (str): Requête de l'utilisateur

    Returns:
        str: Phrase pertinante.
  """
  query = preprocess(query)
  max_similarity = 0
  most_relevant_sentence = ""
  for sentence in corpus:
      similarity = len(set(query).intersection(sentence)) / float(len(set(query).union(sentence)))
      if similarity > max_similarity:
          max_similarity = similarity
          most_relevant_sentence = " ".join(sentence)
  return most_relevant_sentence


def chatbot(question):
  """
    Cette fonction prend la question de l'utilisateur et renvoie la réponse appropriée.

    Args:
        question (str): RequQuestion de l'utilisateur

    Returns:
        str: Réponse su ChatBot.
  """
  most_relevant_sentence = get_most_relevant_sentence(question)
  return most_relevant_sentence

#def main():
st.title("Chatbot")
st.write("Salut!!! Je suis Root👽 votre nouveau ChatBot. En quoi puis-je vous aidez ?")

# Question de l'utilisateur
question = st.text_input("Moi# ")

if st.button("Envoyer"):
    response = chatbot(question)
    st.write("Chatbot@root:~$ " + response)
#if __name__ == "__main__":
    #main()

Writing app.py


In [29]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.921s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [30]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.85.226.42
npx: installed 22 in 2.607s
your url is: https://short-crabs-unite.loca.lt
